# multiprocessing module

The multiprocessing module provides the ability to create processes in the same way as threads from the threading module. Thus, it is possible to bypass the GIL and get real parallel work.

Let's look at an example of how to create a new process:

In [1]:
import os
from multiprocessing import Process, current_process
def foo(number):
    proc_name = current_process().name
    print('{0} {1}'.format(
        number, proc_name))
if __name__ == '__main__':
    random_numbers = [5, 10, 15, 20, 25]
    process_list = []
    proc = Process(target=foo, args=(5,))
    for index, number in enumerate(random_numbers):
        proc = Process(target=foo, args=(number,))
        process_list.append(proc)
        proc.start()   
    proc = Process(target=foo, name='Test', args=(2,))
    proc.start()
    process_list.append(proc)
    for proc in process_list:
        proc.join()

Here we create a new process using the Process class, start it, and in a for loop tell the main Python process to wait until all the child processes we created earlier have terminated.

The Process class takes as arguments:

1. Target is a function that will be executed when the process starts.
2. Name is the name of the process available through the current_process() function.
3. args are the arguments to the target() function.

Just like with threads, processes support Lock to block access to resources.

In [2]:
from multiprocessing import Process, Lock, current_process
def print_function(item, lock):
    lock.acquire()
    try:
       print(item, current_process())
    finally:
        lock.release()
if __name__ == '__main__':
    lock = Lock()
    items = ['test1', 'test2', "test3"]    
    for item in items:
        p = Process(target=print_function, args=(item, lock))
        p.start()

From the example, we can see that, thanks to Lock, the processes work with the function in turn.

An analogue of the ThreadPoolExecutor is the Pool class, which allows you to run several processes at the same time.

In [3]:
from multiprocessing import Pool
def calc(number):
    return number * 2
if __name__ == '__main__':
    numbers = [5, 10, 20]
    pool = Pool(processes=3)
    print(pool.map(calc, numbers))

We create an instance of the Pool class, tell processes that we want to create 3 processes, and then using pool.map() we pass a function to execute and a list of numbers, where each of the elements of the list will subsequently be input to the doubler() function.

For communication between processes, you can use the already familiar Queue class, which is also implemented in the multiprocessing module

In [ ]:
from multiprocessing import Process, Queue
stop_number = -1 
def task_creator(data, q):
    for item in data:
        q.put(item)
def consmuer(q):
    while True:
        data = q.get()
        print('data: {}'.format(data))
        processed = data * 2
        print(processed)
        if data is stop_number:
            break
if __name__ == '__main__':
    q = Queue()
    data = [5, 10, 13, -1]   
    process_one = Process(target=task_creator, args=(data, q))
    process_two = Process(target=consmuer, args=(q,))
    process_one.start()
    process_two.start()
    q.close()
    q.join_thread()
    process_one.join()
    process_two.join()

Here we create two processes and a queue. One process puts data into the queue at startup, and the other reads it and displays it on the screen.

Queues are convenient when you need to link several processes together, for example, some are queued, others are processed.

But the multiprocessing module also has a Pipe class, which allows you to connect only two processes.

In [ ]:
import multiprocessing  
def sender(conn, msgs):
    for msg in msgs:
        conn.send(msg)
        print("Sent the message: {}".format(msg))
    conn.close()

def receiver(conn):
    while 1:
        msg = conn.recv()
        if msg == "END":
            break
        print("Received the message: {}".format(msg))

if __name__ == "__main__":
    msgs = ["START", "END"]
    parent_conn, child_conn = multiprocessing.Pipe()
    p1 = multiprocessing.Process(target=sender, args=(parent_conn,msgs))
    p2 = multiprocessing.Process(target=receiver, args=(child_conn,))  
    p1.start()
    p2.start() 
    p1.join()
    p2.join()

In this example, we create two processes, connect them using Pipe, and organize a simple transfer of text messages from one to another.

I would like to note that Pipe returns two objects - parent_conn and child_conn.

parent_conn is an object that sends data through Pipe.

child_conn is an object that receives data from Pipe.

Pipe is useful when one process is running in the background, for example, monitoring network availability, and another process wants to go to some address and asks another process about network availability, and then makes a decision based on this.